In [1]:
import datetime
import gc
import lightgbm
import lightgbm as lgb
# import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from itertools import compress
# import seaborn as sns
import time
import warnings
import optuna 
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import RFECV
from boruta import BorutaPy

warnings.simplefilter(action='ignore')

FEATS_EXCLUDED = ['first_active_month', 'target', 'card_id', 'outliers',
                  'hist_purchase_date_max', 'hist_purchase_date_min', 'hist_card_id_size',
                  'auth_purchase_date_max', 'auth_purchase_date_min', 'auth_card_id_size',
                  'new_purchase_date_max', 'new_purchase_date_min', 'new_card_id_size',
                  'OOF_PRED', 'month_0']

In [2]:
# rmse
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns
    
# Display/plot feature importance
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]

    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances.png')

# reduce memory
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

## Preprocessing train-test

In [3]:
# load csv
train_df = reduce_mem_usage(pd.read_csv('train.csv', index_col=['card_id']))
test_df = reduce_mem_usage(pd.read_csv('test.csv', index_col=['card_id']))

# outlier
train_df['outliers'] = 0
train_df.loc[train_df['target'] < -30, 'outliers'] = 1

# set target as nan
test_df['target'] = np.nan

# merge
df = train_df.append(test_df)

del train_df, test_df
gc.collect()

# to datetime
df['first_active_month'] = pd.to_datetime(df['first_active_month'])

# datetime features
df['quarter'] = df['first_active_month'].dt.quarter
df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days

df['days_feature1'] = df['elapsed_time'] * df['feature_1']
df['days_feature2'] = df['elapsed_time'] * df['feature_2']
df['days_feature3'] = df['elapsed_time'] * df['feature_3']

df['days_feature1_ratio'] = df['feature_1'] / df['elapsed_time']
df['days_feature2_ratio'] = df['feature_2'] / df['elapsed_time']
df['days_feature3_ratio'] = df['feature_3'] / df['elapsed_time']

# one hot encoding
df, cols = one_hot_encoder(df, nan_as_category=False)

for f in ['feature_1','feature_2','feature_3']:
    order_label = df.groupby([f])['outliers'].mean()
    df[f] = df[f].map(order_label)

df['feature_sum'] = df['feature_1'] + df['feature_2'] + df['feature_3']
df['feature_mean'] = df['feature_sum']/3
df['feature_max'] = df[['feature_1', 'feature_2', 'feature_3']].max(axis=1)
df['feature_min'] = df[['feature_1', 'feature_2', 'feature_3']].min(axis=1)
df['feature_var'] = df[['feature_1', 'feature_2', 'feature_3']].std(axis=1)

Memory usage after optimization is: 4.04 MB
Decreased by 56.2%
Memory usage after optimization is: 2.24 MB
Decreased by 52.5%


## Preprocess merchants

In [4]:
merchants_df = pd.read_csv('merchants.csv')

# Fill NA
merchants_df['category_2'].fillna(1.0, inplace = True)
merchants_df['avg_sales_lag3'].replace(-1, np.nan, inplace = True)
merchants_df['avg_sales_lag6'].replace(-1, np.nan, inplace = True)
merchants_df['avg_sales_lag12'].replace(-1, np.nan, inplace = True)

# Do some preprocessing
merchants_df.drop(['city_id', 'state_id', 'subsector_id', 'category_1', 'category_2'], 1, inplace = True)
merchants_df['category_4'] = merchants_df['category_4'].map({'Y': 1, 'N': 0})

In [5]:
# Merchant agg features
#     aggs['numerical_1'] = ['mean','min','max', 'std']
#     aggs['numerical_2'] = ['mean','min','max', 'std']
#     aggs['most_recent_sales_range'] = ['mean','min','max', 'std']
#     aggs['most_recent_purchases_range'] = ['mean','min','max', 'std']
#     aggs['avg_sales_lag3'] = ['mean','min','max', 'std']
#     aggs['avg_purchases_lag3'] = ['mean','min','max', 'std']
#     aggs['active_months_lag3'] = ['mean','min','max', 'std']
#     aggs['avg_sales_lag6'] = ['mean','min','max', 'std']
#     aggs['avg_purchases_lag6'] = ['mean','min','max', 'std']
#     aggs['active_months_lag6'] = ['mean','min','max', 'std']
#     aggs['avg_sales_lag12'] = ['mean','min','max', 'std']
#     aggs['avg_purchases_lag12'] = ['mean','min','max', 'std']
#     aggs['active_months_lag12'] = ['mean','min','max', 'std']

## Preprocessing historical and new transactions

In [6]:
historical_transactions = pd.read_csv('historical_transactions.csv')
new_transactions = pd.read_csv('new_merchant_transactions.csv')

In [7]:
authorized_transactions = historical_transactions[historical_transactions['authorized_flag'] == 'Y']
historical_transactions = historical_transactions[historical_transactions['authorized_flag'] == 'N']

In [8]:
def create_aggregates_1(df, df_name = 'hist'):
    print("Create aggregates 1")
    
    # fillna
    df['category_2'].fillna(1.0,inplace = True)
    df['category_3'].fillna('A',inplace = True)
    df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace = True)
    df['installments'].replace(-1, np.nan, inplace = True)
    
    # create dummies
    df = pd.get_dummies(df, columns = ['category_2', 'category_3'])
    
    # trim
    df['purchase_amount'] = df['purchase_amount'].apply(lambda x: min(x, 0.8))

    # Y/N to 1/0
    df['authorized_flag'] = df['authorized_flag'].map({'Y': 1, 'N': 0}).astype(int)
    df['category_1'] = df['category_1'].map({'Y': 1, 'N': 0}).astype(int)
#     df['category_3'] = df['category_3'].map({'A': 0, 'B': 1, 'C': 2})

    # datetime features
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['month'] = df['purchase_date'].dt.month
    df['day'] = df['purchase_date'].dt.day
    df['hour'] = df['purchase_date'].dt.hour
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['weekday'] = df['purchase_date'].dt.weekday
    df['weekend'] = (df['purchase_date'].dt.weekday >=5).astype(int)

    # additional features
    df['price'] = df['purchase_amount'] / df['installments']

    #Christmas : December 25 2017
    df['Christmas_Day_2017']=(pd.to_datetime('2017-12-25')-df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
    #Mothers Day: May 14 2017
    df['Mothers_Day_2017']=(pd.to_datetime('2017-06-04') - df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
    #fathers day: August 13 2017
    df['fathers_day_2017']=(pd.to_datetime('2017-08-13')-df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
    #Childrens day: October 12 2017
    df['Children_day_2017']=(pd.to_datetime('2017-10-12')-df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
    #Valentine's Day : 12th June, 2017
    df['Valentine_Day_2017']=(pd.to_datetime('2017-06-12')-df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
    #Black Friday : 24th November 2017
    df['Black_Friday_2017']=(pd.to_datetime('2017-11-24') - df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)

    #2018
    #Mothers Day: May 13 2018
    df['Mothers_Day_2018'] = (pd.to_datetime('2018-05-13') - df['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)

    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']

    # additional features
    df['duration'] = df['purchase_amount'] * df['month_diff']
    df['amount_month_ratio'] = df['purchase_amount'] / df['month_diff']
    
    col_unique = ['subsector_id', 'merchant_id', 'merchant_category_id']
    col_seas = ['month', 'hour', 'weekofyear', 'weekday', 'day']

    aggs = {}
    for col in col_unique:
        aggs[col] = ['nunique']

    for col in col_seas:
        aggs[col] = ['nunique', 'mean', 'min', 'max']

    aggs['purchase_amount'] = ['sum','max','min','mean','var','skew']
    aggs['installments'] = ['sum','max','mean','var','skew']
    aggs['purchase_date'] = ['max','min']
    aggs['month_lag'] = ['max','min','mean','var','skew']
    aggs['month_diff'] = ['max','min','mean','var','skew']
    aggs['authorized_flag'] = ['mean']
    aggs['weekend'] = ['mean']
    aggs['weekday'] = ['mean']
    aggs['day'] = ['nunique', 'mean', 'min']
    aggs['category_1'] = ['sum','mean','max','min']
    aggs['category_2_1.0']: ['sum','mean','max','min']
    aggs['category_2_2.0']: ['sum','mean','max','min']
    aggs['category_2_3.0']: ['sum','mean','max','min']
    aggs['category_2_4.0']: ['sum','mean','max','min']
    aggs['category_2_5.0']: ['sum','mean','max','min']
    aggs['category_3_A']: ['sum','mean','max','min']
    aggs['category_3_B']: ['sum','mean','max','min']
    aggs['category_3_C']: ['sum','mean','max','min']
    aggs['card_id'] = ['size','count']
    aggs['price'] = ['sum','mean','max','min','var']
    aggs['Christmas_Day_2017'] = ['mean']
    aggs['Mothers_Day_2017'] = ['mean']
    aggs['fathers_day_2017'] = ['mean']
    aggs['Children_day_2017'] = ['mean']
    aggs['Valentine_Day_2017'] = ['mean']
    aggs['Black_Friday_2017'] = ['mean']
    aggs['Mothers_Day_2018'] = ['mean']
    aggs['duration']=['mean','min','max','var','skew']
    aggs['amount_month_ratio'] = ['mean','min','max','var','skew']

    for col in ['category_2_1.0', 'category_2_2.0', 'category_2_3.0', 
                'category_2_4.0', 'category_2_5.0', 'category_3_A', 'category_3_B', 'category_3_C']:
        df[col + '_mean'] = df.groupby([col])['purchase_amount'].transform('mean')
        df[col + '_min'] = df.groupby([col])['purchase_amount'].transform('min')
        df[col + '_max'] = df.groupby([col])['purchase_amount'].transform('max')
        df[col + '_sum'] = df.groupby([col])['purchase_amount'].transform('sum')
        aggs[col + '_mean'] = ['mean']
    
    df = df.reset_index().groupby('card_id').agg(aggs)

    # change column name
    df.columns = pd.Index([e[0] + "_" + e[1] for e in df.columns.tolist()])
    df.columns = [df_name + '_'+ c for c in df.columns]

    df[df_name + '_purchase_date_diff'] = (df[df_name + '_purchase_date_max'] - df[df_name + '_purchase_date_min']).dt.days
    df[df_name + '_purchase_date_average'] = df[df_name + '_purchase_date_diff']/df[df_name + '_card_id_size']
    df[df_name + '_purchase_date_uptonow'] = (datetime.datetime.today() - df[df_name + '_purchase_date_max']).dt.days
    df[df_name + '_purchase_date_uptomin'] = (datetime.datetime.today() - df[df_name + '_purchase_date_min']).dt.days
    
    return df

In [18]:
def create_aggregates_2(df, field1):
    print("Create aggregates 2")
    
    # Fill NA
    df['category_2'].fillna(1.0,inplace = True)
    df['category_3'].fillna('A',inplace = True)
    df['installments'].replace(-1, np.nan, inplace = True)
    
    # create dummies
    df = pd.get_dummies(df, columns = ['category_2', 'category_3'])
    df['category_1'] = df['category_1'].map({'Y': 1, 'N': 0}).astype(int)
    
    aggs = {}
    aggs['purchase_amount'] = ['count', 'sum', 'mean', 'min', 'max', 'std', 'skew']
    aggs['installments'] = ['count', 'sum', 'mean', 'min', 'max', 'std', 'skew']
    aggs['category_1'] = ['sum','mean','max','min']
    aggs['category_2_1.0']: ['sum','mean','max','min']
    aggs['category_2_2.0']: ['sum','mean','max','min']
    aggs['category_2_3.0']: ['sum','mean','max','min']
    aggs['category_2_4.0']: ['sum','mean','max','min']
    aggs['category_2_5.0']: ['sum','mean','max','min']
    aggs['category_3_A']: ['sum','mean','max','min']
    aggs['category_3_B']: ['sum','mean','max','min']
    aggs['category_3_C']: ['sum','mean','max','min']
        
    intermediate_group = df.groupby(['card_id', field1]).agg(aggs)
    intermediate_group.columns = [field1 + '_' + '_'.join(col).strip() for col in intermediate_group.columns.values]
    intermediate_group.reset_index(inplace = True)

    final_group = intermediate_group.groupby('card_id').agg(['mean', 'std', 'min', 'max'])
    final_group.columns = [field1 + '_' + '_'.join(col).strip() for col in final_group.columns.values]
    final_group.reset_index(inplace = True)
    
    return final_group

In [10]:
def create_aggregates_3(df, field1, field2):
    print("Create aggregates 3")
    
    t = df.groupby(['card_id', field1])[field2].mean()
    u = pd.DataFrame(t).reset_index().groupby('card_id')[field2].agg(['mean', 'min', 'max', 'std'])
    u.columns = [field1 + '_' + field2 + '_' + col for col in u.columns.values]
    u.reset_index(inplace = True)
    return u

## Create historical transactions aggregates

In [11]:
hist_df_processed_1 = reduce_mem_usage(create_aggregates_1(historical_transactions))

Create aggregates 1
Memory usage after optimization is: 50.75 MB
Decreased by 71.6%


## Create new transactions aggregates

In [12]:
new_df_processed_1 = reduce_mem_usage(create_aggregates_1(new_transactions, df_name = 'new'))
new_df_processed_2 = reduce_mem_usage(create_aggregates_3(new_transactions, 'category_1', 'purchase_amount'))
new_df_processed_2 = reduce_mem_usage(new_df_processed_2.merge(create_aggregates_3(new_transactions, 'installments', 'purchase_amount'),
                                            on = 'card_id', how = 'left'))
new_df_processed_2 = reduce_mem_usage(new_df_processed_2.merge(create_aggregates_3(new_transactions, 'city_id', 'purchase_amount'),
                                            on = 'card_id', how ='left'))
new_df_processed_2 = reduce_mem_usage(new_df_processed_2.merge(create_aggregates_3(new_transactions, 'category_1', 'installments'),
                                            on = 'card_id', how ='left'))

Create aggregates 1
Memory usage after optimization is: 52.27 MB
Decreased by 72.2%
Create aggregates 3
Memory usage after optimization is: 4.43 MB
Decreased by 60.0%
Create aggregates 3
Memory usage after optimization is: 8.85 MB
Decreased by 42.9%
Create aggregates 3
Memory usage after optimization is: 11.06 MB
Decreased by 37.5%
Create aggregates 3
Memory usage after optimization is: 13.28 MB
Decreased by 33.3%


## Create authorized transactions aggregates

In [20]:
# auth_df_processed_1 = reduce_mem_usage(create_aggregates_1(authorized_transactions, df_name = 'auth'))
auth_df_processed_2 = reduce_mem_usage(create_aggregates_2(authorized_transactions, field1 = 'month_lag'))
auth_df_processed_2 = reduce_mem_usage(auth_df_processed_2.merge(create_aggregates_2(authorized_transactions, 'city_id'),
                                        on = 'card_id', how = 'left'))
auth_df_processed_2 = reduce_mem_usage(auth_df_processed_2.merge(create_aggregates_2(authorized_transactions, 'state_id'),
                                        on = 'card_id', how = 'left'))
auth_df_processed_2 = reduce_mem_usage(auth_df_processed_2.merge(create_aggregates_2(authorized_transactions, 'subsector_id'),
                                        on = 'card_id', how = 'left'))

Create aggregates 2
Memory usage after optimization is: 47.81 MB
Decreased by 75.0%
Create aggregates 2
Memory usage after optimization is: 96.24 MB
Decreased by 59.7%
Create aggregates 2
Memory usage after optimization is: 141.57 MB
Decreased by 50.3%
Create aggregates 2
Memory usage after optimization is: 186.90 MB
Decreased by 43.4%


## Join the dataframes

In [22]:
# Join with historical transactions
df1 = pd.merge(df, hist_df_processed_1, on = 'card_id', how = 'left')

# Join with new transactions
df2 = pd.merge(df1, new_df_processed_1, on = 'card_id', how = 'left')
df2 = pd.merge(df2, new_df_processed_2, on = 'card_id', how = 'left')

# Join with authorized transactions
df2 = pd.merge(df2, auth_df_processed_1, on = 'card_id', how = 'left')
df2 = pd.merge(df2, auth_df_processed_2, on = 'card_id', how = 'left')

df2 = reduce_mem_usage(df2)
del df1

Memory usage after optimization is: 395.52 MB
Decreased by 25.1%


## Creating additional features

In [23]:
# additional features
def additional_features(df):
    df['hist_first_buy'] = (df['hist_purchase_date_min'] - df['first_active_month']).dt.days
    df['hist_last_buy'] = (df['hist_purchase_date_max'] - df['first_active_month']).dt.days
    df['new_first_buy'] = (df['new_purchase_date_min'] - df['first_active_month']).dt.days
    df['new_last_buy'] = (df['new_purchase_date_max'] - df['first_active_month']).dt.days
    df['auth_first_buy'] = (df['auth_purchase_date_min'] - df['first_active_month']).dt.days
    df['auth_last_buy'] = (df['auth_purchase_date_max'] - df['first_active_month']).dt.days

    date_features = ['hist_purchase_date_max','hist_purchase_date_min',
                   'new_purchase_date_max', 'new_purchase_date_min', 
                     'auth_purchase_date_max', 'auth_purchase_date_min']

    for f in date_features:
        df[f] = df[f].astype(np.int64) * 1e-9

    df['card_id_total'] = df['new_card_id_size'] + df['hist_card_id_size'] + df['auth_card_id_size']
    df['card_id_cnt_total'] = df['new_card_id_count'] + df['hist_card_id_count'] + df['auth_card_id_count']
    df['card_id_cnt_ratio'] = df['new_card_id_count']/df['hist_card_id_count']
    df['purchase_amount_total'] = df['new_purchase_amount_sum'] + df['hist_purchase_amount_sum'] + df['auth_purchase_amount_sum']
    df['purchase_amount_mean'] = df['new_purchase_amount_mean'] + df['hist_purchase_amount_mean'] + df['auth_purchase_amount_mean']
    df['purchase_amount_max'] = df['new_purchase_amount_max'] + df['hist_purchase_amount_max'] + df['auth_purchase_amount_max']
    df['purchase_amount_min'] = df['new_purchase_amount_min'] + df['hist_purchase_amount_min'] + df['auth_purchase_amount_min']
    df['purchase_amount_ratio'] = df['new_purchase_amount_sum']/df['hist_purchase_amount_sum']
    df['month_diff_mean'] = df['new_month_diff_mean'] + df['hist_month_diff_mean'] + df['auth_month_diff_mean']
    df['month_diff_ratio'] = df['new_month_diff_mean']/df['hist_month_diff_mean']
    df['month_lag_mean'] = df['new_month_lag_mean'] + df['hist_month_lag_mean'] + df['auth_month_lag_mean']
    df['month_lag_max'] = df['new_month_lag_max'] + df['hist_month_lag_max'] + df['auth_month_lag_max']
    df['month_lag_min'] = df['new_month_lag_min'] + df['hist_month_lag_min'] + df['auth_month_lag_min']
    df['category_1_mean'] = df['new_category_1_mean'] + df['hist_category_1_mean'] + df['auth_category_1_mean']
    df['installments_total'] = df['new_installments_sum'] + df['hist_installments_sum'] + df['auth_installments_sum']
    df['installments_mean'] = df['new_installments_mean'] + df['hist_installments_mean'] + df['auth_installments_mean']
    df['installments_max'] = df['new_installments_max'] + df['hist_installments_max'] + df['auth_installments_max']
    df['installments_ratio'] = df['new_installments_sum']/df['hist_installments_sum']
    df['price_total'] = df['purchase_amount_total'] / df['installments_total']
    df['price_mean'] = df['purchase_amount_mean'] / df['installments_mean']
    df['price_max'] = df['purchase_amount_max'] / df['installments_max']
    df['duration_mean'] = df['new_duration_mean'] + df['hist_duration_mean'] + df['auth_duration_mean']
    df['duration_min'] = df['new_duration_min'] + df['hist_duration_min'] + df['auth_duration_min']
    df['duration_max'] = df['new_duration_max'] + df['hist_duration_max'] + df['auth_duration_max']
    df['amount_month_ratio_mean'] = df['new_amount_month_ratio_mean'] + df['hist_amount_month_ratio_mean'] + df['auth_amount_month_ratio_mean']
    df['amount_month_ratio_min'] = df['new_amount_month_ratio_min'] + df['hist_amount_month_ratio_min'] + df['auth_amount_month_ratio_min']
    df['amount_month_ratio_max'] = df['new_amount_month_ratio_max'] + df['hist_amount_month_ratio_max'] + df['auth_amount_month_ratio_max']
    df['new_CLV'] = df['new_card_id_count'] * df['new_purchase_amount_sum'] / df['new_month_diff_mean']
    df['hist_CLV'] = df['hist_card_id_count'] * df['hist_purchase_amount_sum'] / df['hist_month_diff_mean']
    df['CLV_ratio'] = df['new_CLV'] / df['hist_CLV']

    return df

In [24]:
df = additional_features(df2)

In [25]:
train_df = reduce_mem_usage(df[df['target'].notnull()])
test_df = reduce_mem_usage(df[df['target'].isnull()])
del df
gc.collect()

Memory usage after optimization is: 262.08 MB
Decreased by 1.8%
Memory usage after optimization is: 160.81 MB
Decreased by 1.6%


0

In [23]:
train_df.to_csv('train_final.csv')
test_df.to_csv('test_final.csv')

OSError: [Errno 28] No space left on device

## Parameter tuning using Optuna

In [3]:
train_df = pd.read_csv('train_final.csv', index_col = 0)
test_df = pd.read_csv('test_final.csv', index_col = 0)

In [4]:
feats = [c for c in train_df.columns if c not in FEATS_EXCLUDED]

In [29]:
def objective(trial):
    lgbm_train = lightgbm.Dataset(train_df[feats],
                                  train_df['target'],
                                  free_raw_data = False)

    params = {'objective': 'regression',
              'metric': 'rmse',
              'verbosity': -1,
              "learning_rate": 0.01,
              'device': 'cpu',
              'seed': 326,
              'boosting_type': trial.suggest_categorical('boosting', ['gbdt', 'dart', 'goss']),
              'num_leaves': trial.suggest_int('num_leaves', 16, 64),
              'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.001, 1),
              'subsample': trial.suggest_uniform('subsample', 0.001, 1),
              'max_depth': trial.suggest_int('max_depth', 5, 20),
              'reg_alpha': trial.suggest_uniform('reg_alpha', 0, 10),
              'reg_lambda': trial.suggest_uniform('reg_lambda', 0, 10),
              'min_split_gain': trial.suggest_uniform('min_split_gain', 0, 10),
              'min_child_weight': trial.suggest_uniform('min_child_weight', 0, 45),
              'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 16, 64)
              }

    if params['boosting_type'] == 'dart':
        params['drop_rate'] = trial.suggest_loguniform('drop_rate', 1e-8, 1.0)
        params['skip_drop'] = trial.suggest_loguniform('skip_drop', 1e-8, 1.0)
    if params['boosting_type'] == 'goss':
        params['top_rate'] = trial.suggest_uniform('top_rate', 0.0, 1.0)
        params['other_rate'] = trial.suggest_uniform('other_rate', 0.0, 1.0 - params['top_rate'])

    folds = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 47)

    clf = lightgbm.cv(params = params,
                      train_set = lgbm_train,
                      metrics = ['rmse'],
                      nfold = 5,
                      folds = folds.split(train_df, train_df['outliers'].values),
                      num_boost_round = 10000,
                      early_stopping_rounds = 200,
                      verbose_eval = 100,
                      seed = 47,
                     )
    gc.collect()
    return clf['rmse-mean'][-1]

study = optuna.create_study()
study.optimize(objective, n_trials = 100)

print('Number of finished trials: {}'.format(len(study.trials)))

print('Best trial:')
trial = study.best_trial

print('  Value: {}'.format(trial.value))

print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

# save result
results_df = study.trials_dataframe()
results_df.to_csv("optuna_result_lgbm.csv")

print('optuna LightGBM finished.')

[100]	cv_agg's rmse: 3.72582 + 0.00422925
[200]	cv_agg's rmse: 3.69962 + 0.0065143
[300]	cv_agg's rmse: 3.68175 + 0.00725018
[400]	cv_agg's rmse: 3.67242 + 0.00818744
[500]	cv_agg's rmse: 3.66661 + 0.00893988
[600]	cv_agg's rmse: 3.66269 + 0.00936276
[700]	cv_agg's rmse: 3.66007 + 0.00976854
[800]	cv_agg's rmse: 3.65817 + 0.0097436
[900]	cv_agg's rmse: 3.6568 + 0.0100087
[1000]	cv_agg's rmse: 3.65582 + 0.0102578
[1100]	cv_agg's rmse: 3.65496 + 0.0103288
[1200]	cv_agg's rmse: 3.65435 + 0.0104332
[1300]	cv_agg's rmse: 3.65393 + 0.0103457
[1400]	cv_agg's rmse: 3.65357 + 0.0105087
[1500]	cv_agg's rmse: 3.65348 + 0.0104898
[1600]	cv_agg's rmse: 3.65326 + 0.0104754
[1700]	cv_agg's rmse: 3.65311 + 0.0103121
[1800]	cv_agg's rmse: 3.65297 + 0.0104211
[1900]	cv_agg's rmse: 3.65287 + 0.0102684
[2000]	cv_agg's rmse: 3.6528 + 0.010304
[2100]	cv_agg's rmse: 3.65288 + 0.0103469


[I 2019-02-02 06:35:47,518] Finished a trial resulted in value: 3.65269021295721. Current best value is 3.65269021295721 with parameters: {'boosting': 'dart', 'num_leaves': 20, 'colsample_bytree': 0.3151591840582045, 'subsample': 0.11994314702366211, 'max_depth': 18, 'reg_alpha': 4.056366261034116, 'reg_lambda': 5.407611980440702, 'min_split_gain': 3.4283982516942313, 'min_child_weight': 24.40741359383279, 'min_data_in_leaf': 21, 'drop_rate': 1.350364025185153e-07, 'skip_drop': 6.005867376416405e-06}.


[100]	cv_agg's rmse: 3.85824 + 0.00280295
[200]	cv_agg's rmse: 3.85921 + 0.00279997


[I 2019-02-02 06:39:15,377] Finished a trial resulted in value: 3.8469424941852863. Current best value is 3.65269021295721 with parameters: {'boosting': 'dart', 'num_leaves': 20, 'colsample_bytree': 0.3151591840582045, 'subsample': 0.11994314702366211, 'max_depth': 18, 'reg_alpha': 4.056366261034116, 'reg_lambda': 5.407611980440702, 'min_split_gain': 3.4283982516942313, 'min_child_weight': 24.40741359383279, 'min_data_in_leaf': 21, 'drop_rate': 1.350364025185153e-07, 'skip_drop': 6.005867376416405e-06}.


[100]	cv_agg's rmse: 3.7187 + 0.00448484
[200]	cv_agg's rmse: 3.68516 + 0.00722349
[300]	cv_agg's rmse: 3.67237 + 0.00780651
[400]	cv_agg's rmse: 3.66484 + 0.00834688
[500]	cv_agg's rmse: 3.66055 + 0.00874634
[600]	cv_agg's rmse: 3.65777 + 0.00914409
[700]	cv_agg's rmse: 3.65586 + 0.00948844
[800]	cv_agg's rmse: 3.65492 + 0.00970714
[900]	cv_agg's rmse: 3.65438 + 0.00962104
[1000]	cv_agg's rmse: 3.65393 + 0.00958777
[1100]	cv_agg's rmse: 3.65353 + 0.00951627
[1200]	cv_agg's rmse: 3.65335 + 0.00940183
[1300]	cv_agg's rmse: 3.65337 + 0.00933289
[1400]	cv_agg's rmse: 3.65324 + 0.00910269
[1500]	cv_agg's rmse: 3.65323 + 0.00902168
[1600]	cv_agg's rmse: 3.65326 + 0.00888517


...

[100]	cv_agg's rmse: 3.71423 + 0.00516271
[200]	cv_agg's rmse: 3.68102 + 0.00782738
[300]	cv_agg's rmse: 3.66849 + 0.00826714
[400]	cv_agg's rmse: 3.66251 + 0.00887427
[500]	cv_agg's rmse: 3.65914 + 0.00929428
[600]	cv_agg's rmse: 3.65675 + 0.00977231
[700]	cv_agg's rmse: 3.65535 + 0.0102977
[800]	cv_agg's rmse: 3.65457 + 0.010343
[900]	cv_agg's rmse: 3.65406 + 0.0103054
[1000]	cv_agg's rmse: 3.6537 + 0.010269
[1100]	cv_agg's rmse: 3.65357 + 0.0103678
[1200]	cv_agg's rmse: 3.65333 + 0.0102358
[1300]	cv_agg's rmse: 3.65327 + 0.0101313
[1400]	cv_agg's rmse: 3.65323 + 0.00993704
[1500]	cv_agg's rmse: 3.65335 + 0.00971583


[I 2019-02-02 16:05:32,774] Finished a trial resulted in value: 3.653158620458928. Current best value is 3.6496669527976686 with parameters: {'boosting': 'gbdt', 'num_leaves': 60, 'colsample_bytree': 0.24179756580718983, 'subsample': 0.8567867067395918, 'max_depth': 6, 'reg_alpha': 8.738366154766943, 'reg_lambda': 1.0223685093186163, 'min_split_gain': 9.592937554877185, 'min_child_weight': 43.61701338047581, 'min_data_in_leaf': 41}.


[100]	cv_agg's rmse: 3.71022 + 0.00373091
[200]	cv_agg's rmse: 3.67671 + 0.00710699
[300]	cv_agg's rmse: 3.66495 + 0.00762744
[400]	cv_agg's rmse: 3.65902 + 0.00806973
[500]	cv_agg's rmse: 3.65592 + 0.00852272
[600]	cv_agg's rmse: 3.65462 + 0.00897874
[700]	cv_agg's rmse: 3.65359 + 0.00897282
[800]	cv_agg's rmse: 3.65323 + 0.00890183
[900]	cv_agg's rmse: 3.65317 + 0.00882384
[1000]	cv_agg's rmse: 3.65313 + 0.00863144


[I 2019-02-02 16:13:03,011] Finished a trial resulted in value: 3.6530207588885615. Current best value is 3.6496669527976686 with parameters: {'boosting': 'gbdt', 'num_leaves': 60, 'colsample_bytree': 0.24179756580718983, 'subsample': 0.8567867067395918, 'max_depth': 6, 'reg_alpha': 8.738366154766943, 'reg_lambda': 1.0223685093186163, 'min_split_gain': 9.592937554877185, 'min_child_weight': 43.61701338047581, 'min_data_in_leaf': 41}.


Number of finished trials: 100
Best trial:
  Value: 3.6496669527976686
  Params: 
    boosting: gbdt
    num_leaves: 60
    colsample_bytree: 0.24179756580718983
    subsample: 0.8567867067395918
    max_depth: 6
    reg_alpha: 8.738366154766943
    reg_lambda: 1.0223685093186163
    min_split_gain: 9.592937554877185
    min_child_weight: 43.61701338047581
    min_data_in_leaf: 41
optuna LightGBM finished.


## Recursive feature selection

In [3]:
params = {'boosting': 'goss',
        'objective': 'regression',
        'metric': 'rmse',
        'learning_rate': 0.01,
        'subsample': 0.9855232997390695,
        'max_depth': 7,
        'top_rate': 0.9064148448434349,
        'num_leaves': 63,
        'min_child_weight': 41.9612869171337,
        'other_rate': 0.0721768246018207,
        'reg_alpha': 9.677537745007898,
        'colsample_bytree': 0.5665320670155495,
        'min_split_gain': 9.820197773625843,
        'reg_lambda': 8.2532317400459,
        'min_data_in_leaf': 21,
        'verbose': -1,
        'seed': int(2**11),
        'bagging_seed': int(2**11),
        'drop_seed': int(2**11)
}

def RMSE(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

lgb_model = lgb.LGBMRegressor(**params)
rfecv = RFECV(estimator = lgb_model,
              step = 5,
              cv = StratifiedKFold(n_splits = 5,
                                   shuffle = False,
                                   random_state = 42).split(train_df, train_df['outliers'].values),
              verbose = 2)

LightGBMError: Scikit-learn is required for this module

In [ ]:
X, y = train_df[feats], train_df['target']
X = X.replace([np.inf, -np.inf], np.nan)
X = X.fillna(X.median())

rfecv.fit(X, y)

In [ ]:
print(' Optimal number of features: %d' % rfecv.n_features_)
selected_features = [f for f, s in zip(feats, rfecv.support_) if s]
print(' The selected features are {}'.format(selected_features))

## Boruta feature selection

In [39]:
n_fold = 5
params = {'boosting': 'rf',
          'max_depth': 6,
          'njobs': 7,
          'n_estimators': 500,
          'colsample_bytree': 0.5926855122509564,
          'random_state': 420,
          'subsample_freq': 1,
          'subsample': 0.632}

lgb_rf = lgb.LGBMRegressor(**params)
boruta_selector = BorutaPy(lgb_rf, n_estimators = 'auto', verbose = 2)

In [41]:
X, y = train_df[feats], train_df['target']
X = X.replace([np.inf, -np.inf], np.nan)
X = X.fillna(-999)
boruta_selector.fit(X.values, y.values)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	346
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	346
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	346
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	346
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	346
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	346
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	346
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	58
Tentative: 	40
Rejected: 	248
Iteration: 	9 / 100
Confirmed: 	58
Tentative: 	40
Rejected: 	248
Iteration: 	10 / 100
Confirmed: 	58
Tentative: 	40
Rejected: 	248
Iteration: 	11 / 100
Confirmed: 	58
Tentative: 	40
Rejected: 	248
Iteration: 	12 / 100
Confirmed: 	62
Tentative: 	36
Rejected: 	248
Iteration: 	13 / 100
Confirmed: 	62
Tentative: 	36
Rejected: 	248
Iteration: 	14 / 100
Confirmed: 	62
Tentative: 	36
Rejected: 	248
Iteration: 	15 / 100
Confirmed: 	62
Tentative: 	35
Rejected: 	249
Iteration: 	16 / 100
Confirmed: 	

BorutaPy(alpha=0.05,
     estimator=LGBMRegressor(boosting='rf', boosting_type='gbdt', class_weight=None,
       colsample_bytree=0.5926855122509564, importance_type='split',
       learning_rate=0.1, max_depth=6, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=222,
       n_jobs=-1, njobs=7, num_leaves=31, objective=None, random_state=535,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=0.632,
       subsample_for_bin=200000, subsample_freq=1),
     max_iter=100, n_estimators='auto', perc=100,
     random_state=<mtrand.RandomState object at 0x7f06cc05be58>,
     two_step=True, verbose=2)

In [42]:
rankings = dict(zip(train_df.columns, boruta_selector.ranking_))
indices = boruta_selector.support_
selected_features = list(compress(list(train_df.columns), indices))

In [43]:
selected_features

['feature_min',
 'feature_var',
 'hist_month_nunique',
 'hist_hour_nunique',
 'hist_weekofyear_mean',
 'hist_weekday_mean',
 'hist_purchase_amount_max',
 'hist_purchase_amount_min',
 'hist_purchase_date_min',
 'hist_month_lag_min',
 'hist_month_diff_min',
 'hist_authorized_flag_mean',
 'hist_category_1_min',
 'hist_duration_max',
 'hist_category_2_2.0_mean_mean',
 'hist_category_3_C_mean_mean',
 'hist_purchase_date_average',
 'new_month_min',
 'new_hour_nunique',
 'new_hour_max',
 'new_weekofyear_min',
 'new_day_nunique',
 'new_day_mean',
 'new_purchase_amount_var',
 'new_installments_sum',
 'new_month_lag_var',
 'new_month_lag_skew',
 'new_price_min',
 'new_price_var',
 'new_Christmas_Day_2017_mean',
 'new_Mothers_Day_2017_mean',
 'new_fathers_day_2017_mean',
 'new_Valentine_Day_2017_mean',
 'new_Black_Friday_2017_mean',
 'new_amount_month_ratio_var',
 'new_amount_month_ratio_skew',
 'new_category_2_1.0_mean_mean',
 'new_category_2_2.0_mean_mean',
 'new_category_2_3.0_mean_mean',
 'ne

In [44]:
X = boruta_selector.transform(X.values)

## Permutation target feature selection

## Validation, modelling and prediction

In [ ]:
num_folds = 5
stratified = True
print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))

# Cross validation model
if stratified:
    folds = StratifiedKFold(n_splits = num_folds, shuffle = False, random_state = 326)
else:
    folds = KFold(n_splits = num_folds, shuffle = True, random_state = 326)
#     folds = GroupKFold(n_splits = num_folds)

# Create arrays and dataframes to store results
oof_preds = np.zeros(train_df.shape[0])
sub_preds = np.zeros(test_df.shape[0])
feature_importance_df = pd.DataFrame()
feats = [f for f in train_df.columns if f not in FEATS_EXCLUDED]

# k-fold
rmse_list = []
for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['outliers'])):
    train_x, train_y = train_df[feats].iloc[train_idx], train_df['target'].iloc[train_idx]
    valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['target'].iloc[valid_idx]

#     train_x, train_y = X[train_idx], y[train_idx]
#     valid_x, valid_y = X[valid_idx], y[valid_idx]

    # set data structure
    lgb_train = lgb.Dataset(train_x,
                            label = train_y,
                            free_raw_data = False)
    lgb_test = lgb.Dataset(valid_x,
                           label = valid_y,
                           free_raw_data = False)

    # params optimized by optuna
    params = {
                'task': 'train',
                'boosting': 'gbdt',
                'objective': 'regression',
                'metric': 'rmse',
                'learning_rate': 0.01,
                'subsample': 0.8567867067395918,
                'max_depth': 6,
                'num_leaves': 60,
                'min_child_weight': 43.61701338047581,
                'other_rate': 0.0721768246018207,
                'reg_alpha': 8.738366154766943,
                'colsample_bytree': 0.24179756580718983,
                'min_split_gain': 9.592937554877185,
                'reg_lambda': 1.0223685093186163,
                'min_data_in_leaf': 41,
                'verbose': -1,
                'seed':420,
                'bagging_seed': 420,
                'drop_seed': 420,
            }

    reg = lgb.train(
                    params,
                    lgb_train,
                    valid_sets = [lgb_train, lgb_test],
                    valid_names = ['train', 'test'],
                    num_boost_round = 10000,
                    early_stopping_rounds = 200,
                    verbose_eval = 100
                    )

    oof_preds[valid_idx] = reg.predict(valid_x, num_iteration = reg.best_iteration)
    sub_preds += reg.predict(test_df[feats], num_iteration = reg.best_iteration) / folds.n_splits

    print('Fold %2d RMSE : %.6f' % (n_fold + 1, rmse(valid_y, oof_preds[valid_idx])))
    rmse_list.append(rmse(valid_y, oof_preds[valid_idx]))
    del reg, train_x, train_y, valid_x, valid_y
    gc.collect()
print("Mean CV Score - " + str(np.mean(rmse_list)))

Starting LightGBM. Train shape: (201917, 628), test shape: (123623, 629)
Training until validation scores don't improve for 200 rounds.
[100]	train's rmse: 3.67125	test's rmse: 3.71497
[200]	train's rmse: 3.59778	test's rmse: 3.67968
[300]	train's rmse: 3.55643	test's rmse: 3.66894
[400]	train's rmse: 3.52738	test's rmse: 3.66367
[500]	train's rmse: 3.50331	test's rmse: 3.66018
[600]	train's rmse: 3.48223	test's rmse: 3.65793
[700]	train's rmse: 3.4623	test's rmse: 3.65698
[800]	train's rmse: 3.44502	test's rmse: 3.65668
[900]	train's rmse: 3.42836	test's rmse: 3.65677
[1000]	train's rmse: 3.41297	test's rmse: 3.65692
[1100]	train's rmse: 3.39748	test's rmse: 3.65689
Early stopping, best iteration is:
[922]	train's rmse: 3.42452	test's rmse: 3.65656
Fold  1 RMSE : 3.656560
Training until validation scores don't improve for 200 rounds.
[100]	train's rmse: 3.66831	test's rmse: 3.71948
[200]	train's rmse: 3.59457	test's rmse: 3.68531
[300]	train's rmse: 3.55286	test's rmse: 3.6738
[400]	t

In [28]:
# save submission file
test_df.loc[:, 'target'] = sub_preds
test_df = test_df.reset_index()
test_df[['card_id', 'target']].to_csv('lgb_650_feats_5_fold_2.csv', index=False)

## Stacking